In [27]:
from scraping import get_soup,find_all,find
import pandas as pd

In [2]:
soup = get_soup(url="https://www.curlingzone.com/rankings.php?task=week&oomid=81&eventyear=2021#1")

In [3]:
soup

<!DOCTYPE html>

<!--[if IE 7]>
<html class="ie ie7" lang="en-US">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" lang="en-US">
<![endif]-->
<!--[if !(IE 7) | !(IE 8)  ]><!-->
<html lang="en-US">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.curlingzone.com/talk/xmlrpc.php" rel="pingback"/>
<meta content="summary_large_image" name="twitter:card">
<meta content="@curlingzone" name="twitter:site">
<meta content="@guertez" name="twitter:creator"/>
<title>CurlingZone – Everything Curling</title>
<link href="//cdnjs.cloudflare.com" rel="dns-prefetch"/>
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://www.curlingzone.com/?feed=rss2" rel="alternate" title="CurlingZone » Feed" type="application/rss+xml"/>
<link href="https://www.curlingzone.com/?feed=comments-rss2" rel="al

In [44]:
def find_all_teams(soup):
    return find_all(soup=soup,name='td',attrs={'data-th':'Team'})
def find_all_ranks(soup):
    return find_all(soup=soup,name='td',attrs={'data-th':'Rank'})
def find_all_totals(soup):
    return find_all(soup=soup,name='td',attrs={'data-th':'Total'})
def find_all_ytds(soup):
    return find_all(soup=soup,name='td',attrs={'data-th':'YTD'})

In [45]:
find_all_ranks(soup)[0:5]

[<td data-th="Rank">1.</td>,
 <td data-th="Rank">2.</td>,
 <td data-th="Rank">3.</td>,
 <td data-th="Rank">4.</td>,
 <td data-th="Rank">5.</td>]

In [46]:
find_all_totals(soup)[0:5]

[<td data-th="Total">483.766</td>,
 <td data-th="Total">445.311</td>,
 <td data-th="Total">395.747</td>,
 <td data-th="Total">347.977</td>,
 <td data-th="Total">343.005</td>]

In [47]:
find_all_ytds(soup)[0:5]

[<td data-th="YTD">0.000</td>,
 <td data-th="YTD">0.000</td>,
 <td data-th="YTD">0.000</td>,
 <td data-th="YTD">0.000</td>,
 <td data-th="YTD">0.000</td>]

In [48]:
def get_team_ranks(tags):
    return {
        
         "Rank":[tag.string.strip() for tag in tags]
        ,"TeamElementHash":[tag.parent.__hash__() for tag in tags]
        
        
    }

def get_team_ranks_from_soup(soup):
    tags = find_all_ranks(soup=soup)
    return get_team_ranks(tags=tags)

def get_team_ranks_frame_from_soup(soup):
    return get_frame(get_team_ranks_from_soup(soup=soup))

def get_team_totals(tags):
    return{
        
         "PointTotal":[tag.string.strip() for tag in tags]
        ,"TeamElementHash":[tag.parent.__hash__() for tag in tags]
        
    }

def get_team_totals_from_soup(soup):
    tags = find_all_totals(soup=soup)
    return get_team_totals(tags=tags)

def get_team_totals_frame_from_soup(soup):
    return get_frame(get_team_totals_from_soup(soup=soup))
    
def get_team_ytds(tags):
    return {
        
         "YTDPoints":[tag.string.strip() for tag in tags]
        ,"TeamElementHash":[tag.parent.__hash__() for tag in tags]
        
        
    }
def get_team_ytds_from_soup(soup):
    tags = find_all_ytds(soup=soup)
    return get_team_ytds(tags=tags)

def get_team_ytds_frame_from_soup(soup):
    return get_frame(get_team_ytds_from_soup(soup=soup))

def get_team_links(tags):
    return {
        
         "TeamLink":[tag.a['href'] for tag in tags if tag.a.string]
        ,"Name":[tag.a.string for tag in tags if tag.a.string]
        ,"TeamElementHash":[tag.parent.__hash__() for tag in tags if tag.a.string]
        
    }

def get_team_links_from_soup(soup):
    tags = find_all_teams(soup=soup)
    return get_team_links(tags=tags)

def get_team_links_frame_from_soup(soup):
    return get_frame(get_team_links_from_soup(soup=soup))

def get_frame(data,index=True):
    
    df = pd.DataFrame(data=data)
    
    if index:
        df=df.set_index(keys='TeamElementHash',drop=True)
        
    return df 

def scrape_all_rankings(url,year=None):
    soup = get_soup(url=url)
    
    team_links = get_team_links_frame_from_soup(soup)
    team_ytds = get_team_ytds_frame_from_soup(soup)
    team_totals = get_team_totals_frame_from_soup(soup)
    team_ranks = get_team_ranks_frame_from_soup(soup)
    
    df = pd.concat([team_links,team_ytds,team_totals,team_ranks],axis=1)
    
    df['Year'] = year

    
    return df

In [49]:
scrape_all_rankings(url="https://www.curlingzone.com/rankings.php?task=week&oomid=81&eventyear=2020#1")

,TeamLink,Name,YTDPoints,PointTotal,Rank,Year
TeamElementHash,,,,,,
-1143695436,team.php?teamid=134539&profileid=&view=OOM&OOM...,Brad Jacobs,460.083,460.083,1.,None
-170019375,team.php?teamid=134149&profileid=&view=OOM&OOM...,John Epping,438.075,438.075,2.,None
-2033196918,team.php?teamid=134658&profileid=&view=OOM&OOM...,Bruce Mouat,343.005,343.005,3.,None
-996801756,team.php?teamid=133900&profileid=&view=OOM&OOM...,Yannick Schwaller,330.559,330.559,4.,None
-1837416361,team.php?teamid=134492&profileid=&view=OOM&OOM...,Brad Gushue,320.844,320.844,5.,None
266977469,team.php?teamid=134333&profileid=&view=OOM&OOM...,Mike McEwen,311.649,311.649,6.,None
-1294030979,team.php?teamid=134858&profileid=&view=OOM&OOM...,Niklas Edin,304.836,304.836,7.,None
-1024578674,team.php?teamid=134525&profileid=&view=OOM&OOM...,Brendan Bottcher,297.973,297.973,8.,None
-760583075,team.php?teamid=134345&profileid=&view=OOM&OOM...,Peter De Cruz,289.136,289.136,9.,None
